In [16]:
from keras.models import Model
from keras.layers import Lambda,Activation,Dense,Conv2D,Input,BatchNormalization,MaxPool2D,Flatten
import keras.backend as K
import numpy as np

In [17]:
input_tensor_1 = Input([32,32,3])  # keras里面的Input是定义shape，并不是真正的input
input_tensor_2 = Input([4,])
input_target = Input([2,])

In [18]:
# loss计算函数
def cus_loss1(y_true,y_pred):
    return K.mean(K.abs(y_true-y_pred))
def cus_loss2(y_true,y_pred):
    return K.mean(K.abs(y_true-y_pred))

In [19]:
# 网络结构
# 第一组
x = BatchNormalization(axis=1)(input_tensor_1)  # 可以加name属性

x = Conv2D(filters=32, kernel_size=(3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Conv2D(filters=32, kernel_size=(3,3), padding='same')(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)

x = Flatten()(x)
x = Dense(units=16)(x)
out2 = Dense(units=2)(x)

# 第二组
y = Dense(units=32)(input_tensor_2)
out1 = Dense(units=2)(y)

# 第三组
z = Dense(units=8)(input_target)
out3 = Dense(units=2)(z)


# 自定义一个loss
loss1 = Lambda(lambda x:cus_loss1(*x),name='loss1')([out2,out1])   # Lambda是自定义层
                                                    # *x代表传入的是一个列表
loss2 = Lambda(lambda x:cus_loss2(*x),name='loss2')([out3,out2])

In [20]:
model = Model(input=[input_tensor_1,input_tensor_2,input_target], output=[out1,out2,out3,loss1,loss2])

D:\1-Anaconda\AnacondaInstall\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [21]:
# # 画出model
# from keras.utils.vis_utils import plot_model
# plot_model(model=model, to_file='model_plain_loss.png', show_shapes=True)

In [22]:
# 获取loss
loss_layer1 = model.get_layer('loss1').output
loss_layer2 = model.get_layer('loss2').output

# 向模型添加loss
model.add_loss(loss_layer1)
model.add_loss(loss_layer2)

In [23]:
# loss + None 表示不需要梯度返回，有多少个out就有多少个none
model.compile(optimizer='sgd',loss=[None,None,None,None,None])

In [29]:
# dataset
def data_gen(number):
    for i in range(number):
        yield [np.random.normal(1,1,size=(1,32,32,3)),np.random.normal(1,1,size=(1,4)),np.random.normal(1,1,size=(1,2))],[]
dataset = data_gen(1000)

In [30]:
dataset.__next__()

([array([[[[ 2.63954217, -0.3445296 ,  0.96035656],
           [ 0.97955626,  2.64003951,  0.60231907],
           [ 0.6131107 ,  1.42697695,  0.20306063],
           ...,
           [ 1.39460354, -0.77776338,  0.66113531],
           [-0.07946318, -0.03178072,  0.59098456],
           [ 1.57143944,  1.17905077,  1.87900025]],
  
          [[-0.70533258,  0.81045542,  1.76881592],
           [ 0.02913281,  0.70037198, -0.04116736],
           [ 2.36989389,  0.25593629,  0.20323929],
           ...,
           [ 2.34460204,  1.31967496,  1.58689233],
           [ 0.10552273,  2.01855264,  0.68046942],
           [ 0.30189685,  0.85026914,  0.13743315]],
  
          [[ 0.57432713,  1.47160032,  1.27445483],
           [ 0.57982915,  1.98223884,  2.24452048],
           [ 1.28236662,  0.26412386,  2.36374971],
           ...,
           [ 3.01568568,  1.88988299,  0.4709379 ],
           [ 1.54834852, -0.06381629, -0.59294323],
           [ 1.42068148,  0.51410032, -1.20015054]],
  
    

In [31]:
train = model.fit_generator(dataset,epochs=10,steps_per_epoch=20)

Epoch 1/10
20/20 [==============================] - 1s 41ms/step - loss: 1.8227
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.7559
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4088
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.2393
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1963
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1665
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1741
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1559
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1224
Epoch 10/10
20/20 [==============================] - 0s 3ms/step - loss: 0.1629
